In [1]:
from google.colab import files
uploaded = files.upload()

Saving FINAL_PERTURBED_DATASET.csv to FINAL_PERTURBED_DATASET.csv


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "deepseek-ai/deepseek-llm-7b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, # make it float32 if you face errors or ping me
    device_map="auto",
    offload_folder="./offload"
)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
from tqdm.notebook import tqdm

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
df = pd.read_csv("FINAL_PERTURBED_DATASET.csv")
print(df.columns.tolist())

Device set to use cuda:0


['id', 'date', 'bias_type', 'normative_framing', 'source', 'policy', 'bias_type_merged', 'policy_perturbed']


**NINE SHOT PERTURBED**

In [4]:
#Perturbed 9 shot
import re
tqdm.pandas()

from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
def classify_bias(policy):
    prompt = [
          {"role": "system",
         "content": (f"You are a policy-analysis assistant.\n"
                     f"Given a policy excerpt, output *exactly one* of these three labels—nothing else, all lower-case:\n"
                     f"- no_bias\n"
                     f"- group_1 (political, criminal_justice, citizenship, disability, education)\n"
                     f"- group_2 \n\n"
                     f"Definitions:\n"
                     f"- *group_2*: policies related to identity, social, and economic status, including:\n"
                     f"• economic: references income, poverty, homelessness, wealth, financial assistance, costs, payments, fees, funding, housing, area living in, etc.\n"
                     f"• racial_cultural: references race, ethnicity, culture, personal beliefs, etc.\n"
                     f"• age: references children, youth, child welfare policies, adults, elderly, seniors, age-related policies, etc.\n"
                     f"• religion: references religious beliefs, religious groups, faith-based accommodations, etc.\n"
                     f"• gender: references women, men, sex, gender identity, use of only one pronoun for a role, sexual harassment, reproductive rights, pregnancy, etc.\n\n"
                     f"- *group_1*: policies related to legal, institutional, or civic systems, including:\n"
                     f"• political: references voting rights, politics, elections, campaigns, government, war, international relations etc.1\n"
                     f"• criminal_justice: references crime, criminals, court, law enforcement, policing, prison, etc.\n"
                     f"• citizenship: references immigration, immigration status, deportation, visas, border control, etc.\n"
                     f"• disability: references physical or mental impairments, accommodations for impairments, accessibility, illness, etc.\n"
                     f"• education: curriculum, degrees, teaching credentials, language proficiency, language required to learn, standardized testing, school admission, etc.\n"
                     f"- *no_bias*: procedural, definitional, administrative, factual, or operational text, like implementation details or definitions.\n\n" #highlighted no_bias
                     f"Examples:\n\n"
                     f"Example 1:\n"
                     f"Policy: The job application process mandates that candidates disclose their age and retirement plans, with younger applicants prioritized for advancement.\n"
                     f"Classification: group_2\n\n"
                     f"Example 2:\n"
                     f"Policy: Tax exemptions will be granted to corporations investing over $5 million in designated opportunity zones.\n"
                     f"Classification: group_2\n\n"
                     f"Example 3:\n"
                     f"Policy: The bill excludes same-sex partners from family leave benefits.\n"
                     f"Classification: group_2\n\n"
                     f"Example 4:\n"
                     f"Policy: Immigrants must demonstrate fluency in English and pass a civic exam to be eligible for naturalization.\n"
                     f"Classification: group_1\n\n"
                     f"Example 5:\n"
                     f"Policy: The policy mandates separate healthcare facilities for individuals with disabilities, citing operational constraints.\n"
                     f"Classification: group_1\n\n"
                     f"Example 6:\n"
                     f"Policy: A bill mandates non-citizens charged with a Class 4 felony shall be designated deportable immediately upon indictment.\n"
                     f"Classification: group_1\n\n"
                     f"Example 7:\n"
                     f"Policy: The committee will meet quarterly to review environmental compliance reports.\n"
                     f"Classification: no_bias\n\n"
                     f"Example 8:\n"
                     f"Policy: Applicants must submit identity verification and a signed consent form for record access.\n"
                     f"Classification: no_bias\n\n"
                     f"Example 9:\n"
                     f"Policy: The funding request was approved following standard procurement guidelines and budgetary review.\n"
                     f"Classification: no_bias\n"
                     f"Decision Rules:\n"
                     f"- Focus on legal or civic status -> group_1.\n"
                     f"- Focus on economic standing or demographic identity -> group_2.\n"
                     f"- Purely procedural/factual -> no_bias.\n\n"
                     )},
        {"role": "user",
         "content": (f"Classify the following excerpt:\n"
                     f"\"\"\"\n"
                     f"{policy}\n"
                     f"\"\"\""
                     f"When uncertain, pick the best matching category. No explanations.\n"
                     f"Return *only* one of: group_1, group_2, no_bias,\n\n"

                )}
              ]


    result = pipe(prompt, max_new_tokens=10, do_sample=False)
    generated_text = result[0]['generated_text']
    predicted = generated_text[-1]["content"].strip().split("\n")[0]
    return predicted

#################################################################################################
# Define valid classes
valid_labels = ["group_2", "group_1", "no_bias"]


# Apply classifier
df['predicted_bias'] = df['policy_perturbed'].progress_apply(classify_bias) #POLICY_PERTURBED

################################################################################################

from sklearn.metrics import classification_report, accuracy_score

#clean predictions
df['predicted_bias'] = df['predicted_bias'].astype(str).str.strip().str.lower()
df['bias_type_merged'] = df['bias_type_merged'].astype(str).str.strip().str.lower()

# Accuracy
df['correct'] = df['bias_type_merged'] == df['predicted_bias']
accuracy = df['correct'].mean()
print(f"\nAccuracy: {accuracy * 100:.2f}%")

# Precision, Recall, F1
print("\nClassification Report:")
print(classification_report(
    df['bias_type_merged'],
    df['predicted_bias'],
    labels=valid_labels,
    digits=3
))

# download
df.to_csv("Deepseek7B_Nine_Shot_Perturbed.csv", index=False)

# If in Colab or Jupyter:
from google.colab import files
files.download("Deepseek7B_Nine_Shot_Perturbed.csv")

Device set to use cuda:0


  0%|          | 0/20694 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Accuracy: 40.09%

Classification Report:
              precision    recall  f1-score   support

     group_2      0.385     0.422     0.402      6969
     group_1      0.407     0.355     0.379      7339
     no_bias      0.414     0.430     0.422      6386

   micro avg      0.401     0.401     0.401     20694
   macro avg      0.402     0.403     0.401     20694
weighted avg      0.402     0.401     0.400     20694



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>